<a href="https://colab.research.google.com/github/simecek/mlprague2024/blob/main/06_QLoRA_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()
# Must install separately since Colab has torch 2.2.1, which breaks packages
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
    !pip install --no-deps packaging ninja flash-attn xformers trl peft accelerate bitsandbytes
else:
    # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-7b-bnb-4bit", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth: Fast Gemma patching release 2024.4
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = True,
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.4 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


<a name="Data"></a>
### Data Prep

We now use the Alpaca dataset from [yahma](https://huggingface.co/datasets/yahma/alpaca-cleaned), which is a filtered version of 52K of the original [Alpaca dataset](https://crfm.stanford.edu/2023/03/13/alpaca.html). You can replace this code section with your own data prep.

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

If you want to use the `ChatML` template for ShareGPT datasets, try our conversational [notebook](https://colab.research.google.com/drive/1Aau3lgPzeZKQ-98h69CCu1UJcvIBLmy2?usp=sharing).

For text completions like novel writing, try this [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing).

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    outputs      = examples["output"]
    texts = []
    for instruction, output in zip(instructions, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
#dataset = load_dataset("yahma/alpaca-cleaned", split = "train")
dataset = load_dataset("hynky/czech-justice-summ-alpaca-short", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

In [ ]:
dataset

Dataset({
    features: ['output', 'instruction', 'text'],
    num_rows: 2015
})

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 8,
        warmup_steps = 5,
        max_steps = 20,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
5.889 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 2,015 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 8
\        /    Total batch size = 8 | Total steps = 20
 "-____-"     Number of trainable parameters = 50,003,968


Step,Training Loss
1,1.893500
2,1.753800
3,2.084200
4,1.790800
5,1.526100
6,1.561400
7,1.452000
8,1.268200
9,1.843000
10,1.624100


In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

581.1083 seconds used for training.
9.69 minutes used for training.
Peak reserved memory = 9.086 GB.
Peak reserved memory for training = 3.197 GB.
Peak reserved memory % of max memory = 61.608 %.
Peak reserved memory for training % of max memory = 21.678 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

In [ ]:
law = """Oddíl 3

Nájem
Pododdíl 1

Obecná ustanovení
§ 2201

Základní ustanovení
Nájemní smlouvou se pronajímatel zavazuje přenechat nájemci věc k dočasnému užívání a nájemce se zavazuje platit za to pronajímateli nájemné.

§ 2202

Předmět nájmu
(1) Pronajmout lze věc nemovitou i nezuživatelnou věc movitou. Pronajmout lze i část nemovité věci; co se dále stanoví o věci, použije se i pro nájem její části.

(2) Pronajmout lze i věc, která v budoucnu teprve vznikne, je-li ji možné dostatečně přesně určit při uzavření nájemní smlouvy.

§ 2203

Je-li pronajatá věc zapsána do veřejného seznamu, zapíše se do veřejného seznamu i nájemní právo, pokud to navrhne vlastník věci nebo s jeho souhlasem nájemce.

§ 2204

(1) Neujednají-li strany dobu trvání nebo den skončení nájmu, platí, že se jedná o nájem na dobu neurčitou.

(2) Ujednají-li strany nájem na dobu určitou delší než padesát let, má se za to, že byl nájem ujednán na dobu neurčitou s tím, že v prvních padesáti letech lze nájem vypovědět jen z ujednaných výpovědních důvodů a v ujednané výpovědní době.

Pronajímatel

§ 2205

Nájemní smlouva pronajímatele zavazuje

a) přenechat věc nájemci tak, aby ji mohl užívat k ujednanému nebo obvyklému účelu,

b) udržovat věc v takovém stavu, aby mohla sloužit tomu užívání, pro které byla pronajata,

c) zajistit nájemci nerušené užívání věci po dobu nájmu.

§ 2206

(1) Pronajímatel odevzdá nájemci věc v ujednané době, jinak v den následující poté, co jej o to nájemce požádá.

(2) Pronajímatel odevzdá nájemci věc se vším, co je třeba k řádnému užívání věci.

§ 2207

(1) Po dobu nájmu provádí běžnou údržbu věci nájemce, ledaže se k ní zavázal pronajímatel. Ostatní údržbu věci a její nezbytné opravy provádí pronajímatel, ledaže se k některému způsobu nebo druhu údržby a k opravě některých vad zavázal nájemce.

(2) Pronajímatel neodpovídá za vadu, o které v době uzavření nájemní smlouvy strany věděly a která nebrání užívání věci.

§ 2208

(1) Oznámí-li nájemce řádně a včas pronajímateli vadu věci, kterou má pronajímatel odstranit, a neodstraní-li pronajímatel vadu bez zbytečného odkladu, takže nájemce může věc užívat jen s obtížemi, má nájemce právo na přiměřenou slevu z nájemného nebo může provést opravu také sám a požadovat náhradu účelně vynaložených nákladů. Ztěžuje-li však vada zásadním způsobem užívání, nebo znemožňuje-li zcela užívání, má nájemce právo na prominutí nájemného nebo může nájem vypovědět bez výpovědní doby.

(2) Nájemce má právo započíst si to, co může podle odstavce 1 žádat od pronajímatele, až do výše nájemného za jeden měsíc; je-li doba nájmu kratší, až do výše nájemného.

(3) Neuplatní-li nájemce právo podle odstavce 1 do šesti měsíců ode dne, kdy vadu zjistil nebo mohl zjistit, soud mu je nepřizná, namítne-li pronajímatel jeho opožděné uplatnění.

§ 2209

Během nájmu pronajímatel nemá právo o své vůli pronajatou věc měnit.

§ 2210

(1) Ukáže-li se během nájmu potřeba provést nezbytnou opravu věci, kterou nelze odložit na dobu po skončení nájmu, musí ji nájemce strpět, i když mu provedení opravy způsobí obtíže nebo omezí užívání věci.

(2) Trvá-li oprava vzhledem k době nájmu dobu nepřiměřeně dlouhou nebo ztěžuje-li oprava užívání věci nad míru obvyklou, má nájemce právo na slevu z nájemného podle doby opravy a jejího rozsahu.

(3) Jedná-li se o takovou opravu, že v době jejího provádění není možné věc vůbec užívat, má nájemce právo, aby mu pronajímatel dočasně poskytl k užívání jinou věc, nebo může nájem vypovědět bez výpovědní doby.

§ 2211

Ohrozí-li třetí osoba nájemce v jeho nájemním právu nebo způsobí-li nájemci porušením nájemního práva újmu, může se ochrany domáhat nájemce sám.

§ 2212

(1) Uplatňuje-li třetí osoba vlastnické nebo jiné právo k věci nebo žádá-li vydání nebo vyklizení věci, nájemce to pronajímateli oznámí; požádá-li o to, poskytne mu pronajímatel ochranu.

(2) Neposkytne-li pronajímatel nájemci dostatečnou ochranu, může nájemce nájem vypovědět bez výpovědní doby.

(3) Bude-li nájemce rušen v užívání věci nebo jinak dotčen jednáním třetí osoby, má právo na přiměřenou slevu z nájemného, pokud takové jednání třetí osoby pronajímateli včas oznámil.

Nájemce

§ 2213

Nájemce je i bez zvláštního ujednání povinen užívat věc jako řádný hospodář k ujednanému účelu, nebo není-li ujednán, k účelu obvyklému, a platit nájemné.

§ 2214

Nájemce oznámí pronajímateli, že věc má vadu, kterou má odstranit pronajímatel, hned poté, kdy ji zjistí nebo kdy ji při pečlivém užívání věci zjistit mohl.

Podnájem

§ 2215

(1) Souhlasí-li pronajímatel, může nájemce zřídit třetí osobě k věci užívací právo; byla-li nájemní smlouva uzavřena v písemné formě, vyžaduje i souhlas pronajímatele písemnou formu.

(2) Zřídí-li nájemce třetí osobě užívací právo k věci bez souhlasu pronajímatele, považuje se to za hrubé porušení nájemcových povinností způsobující pronajímateli vážnější újmu.

(3) Užívací právo lze třetí osobě zřídit jen na dobu nájmu věci; k odchylnému ujednání se nepřihlíží.

§ 2216

Umožní-li nájemce užívat věc třetí osobě, odpovídá pronajímateli za jednání této osoby stejně, jako kdyby věc užíval sám.

Nájemné

§ 2217

(1) Nájemné se platí v ujednané výši, a není-li ujednána, platí se ve výši obvyklé v době uzavření nájemní smlouvy s přihlédnutím k nájemnému za nájem obdobných věcí za obdobných podmínek.

(2) Má-li být nájemné podle ujednání stran plněno jinak než v penězích, je rozhodná majetková hodnota poskytovaného plnění vyjádřená v penězích.

§ 2218

Nájemné se platí měsíčně pozadu.

Další práva a povinnosti stran

§ 2219

(1) Oznámí-li to pronajímatel předem v přiměřené době, umožní mu nájemce v nezbytném rozsahu prohlídku věci, jakož i přístup k ní nebo do ní za účelem provedení potřebné opravy nebo údržby věci. Předchozí oznámení se nevyžaduje, je-li nezbytné zabránit škodě nebo hrozí-li nebezpečí z prodlení.

(2) Vzniknou-li nájemci činností pronajímatele podle odstavce 1 obtíže, které nejsou jen nepodstatné, má nájemce právo na slevu z nájemného.

§ 2220

(1) Nájemce má právo provést změnu věci jen s předchozím souhlasem pronajímatele; byla-li nájemní smlouva uzavřena v písemné formě, vyžaduje i souhlas pronajímatele písemnou formu. Změnu věci provádí nájemce na svůj náklad; dojde-li změnou věci k jejímu zhodnocení, pronajímatel se s nájemcem při skončení nájmu vyrovná podle míry zhodnocení.

(2) Provede-li nájemce změnu věci bez souhlasu pronajímatele, uvede věc do původního stavu, jakmile o to pronajímatel požádá, nejpozději však při skončení nájmu věci. Neuvede-li nájemce na žádost pronajímatele věc do původního stavu, může pronajímatel nájem vypovědět bez výpovědní doby.

Změna vlastnictví

§ 2221

(1) Změní-li se vlastník věci, přejdou práva a povinnosti z nájmu na nového vlastníka.

(2) Převedl-li pronajímatel vlastnické právo k věci, nejsou pro nového vlastníka závazná ujednání o pronajímatelových povinnostech, které zákon nestanoví. To neplatí, pokud nový vlastník o těchto ujednáních věděl.

§ 2222

(1) Strana nemá právo vypovědět nájem jen proto, že se změnil vlastník věci. Při opačném ujednání má pronajímatel právo nájem vypovědět do tří měsíců poté, co se dozvěděl nebo musel dozvědět, kdo je nájemcem, a nájemce do tří měsíců poté, co se o změně vlastníka dozvěděl.

(2) Neměl-li nový vlastník rozumný důvod pochybovat, že kupuje věc, která není pronajata, má právo vypovědět nájem do tří měsíců poté, co se dozvěděl nebo musel dozvědět, že je věc pronajata a kdo je nájemcem. Nájemcova práva vůči osobě, se kterou nájemní smlouvu uzavřel, nejsou dotčena.

(3) Jedná-li se o nemovitou věc, je výpovědní doba tříměsíční. Jedná-li se o movitou věc, je výpovědní doba jednoměsíční.

§ 2223

Strana, která nájem vypoví, poskytne druhé straně přiměřené odstupné.

§ 2224

Byl-li pronajat byt, ve kterém nájemce bydlí, nemá pronajímatel právo nájem vypovědět z důvodu změny vlastnictví. K opačnému ujednání se nepřihlíží.

Skončení nájmu

§ 2225

(1) Při skončení nájmu odevzdá nájemce pronajímateli věc v místě, kde ji převzal, a v takovém stavu, v jakém byla v době, kdy ji převzal, s přihlédnutím k obvyklému opotřebení při řádném užívání, ledaže věc zanikla nebo se znehodnotila; odevzdáním se rozumí i předání vyklizené nemovité věci. Byl-li při odevzdání věci nájemci pořízen zápis obsahující popis věci, přihlédne se při odevzdání věci pronajímateli také k němu.

(2) Při odevzdání věci si nájemce oddělí a vezme vše, co do věci vložil nebo na ni vnesl vlastním nákladem, je-li to možné a nezhorší-li se tím podstata věci nebo neztíží-li se tím nepřiměřeně její užívání.

§ 2226

(1) Zanikne-li věc během doby nájmu, nájem skončí.

(2) Zanikne-li věc během doby nájmu zčásti, má nájemce právo buď na přiměřenou slevu z nájemného, anebo může nájem vypovědět bez výpovědní doby.

§ 2227

Stane-li se věc nepoužitelnou k ujednanému účelu, nebo není-li ujednán, k účelu obvyklému, a to z důvodů, které nejsou na straně nájemce, má nájemce právo nájem vypovědět bez výpovědní doby.

§ 2228

(1) Užívá-li nájemce věc takovým způsobem, že se opotřebovává nad míru přiměřenou okolnostem nebo že hrozí zničení věci, vyzve ho pronajímatel, aby věc užíval řádně, dá mu přiměřenou lhůtu k nápravě a upozorní jej na možné následky neuposlechnutí výzvy. Výzva vyžaduje písemnou formu a musí být nájemci doručena.

(2) Neuposlechne-li nájemce výzvy podle odstavce 1, má pronajímatel právo nájem vypovědět bez výpovědní doby.

(3) Hrozí-li však v případě uvedeném v odstavci 1 naléhavě vážné nebezpečí z prodlení, má pronajímatel právo nájem vypovědět bez výpovědní doby, aniž nájemce vyzval k nápravě.

(4) Pronajímatel má právo postupovat stejně, jak je uvedeno v odstavcích 1 a 2, nezaplatí-li nájemce nájemné ani do splatnosti příštího nájemného.

§ 2229

Nájem ujednaný na dobu určitou může každá ze stran vypovědět jen v případě, že ve smlouvě byly zároveň ujednány důvody výpovědi a výpovědní doba.

§ 2230

(1) Užívá-li nájemce věc i po uplynutí nájemní doby a pronajímatel ho do jednoho měsíce nevyzve, aby mu věc odevzdal, platí, že nájemní smlouva byla znovu uzavřena za podmínek ujednaných původně. Byla-li původně nájemní doba delší než jeden rok, platí, že nyní byla uzavřena na jeden rok; byla-li kratší než jeden rok, platí, že nyní byla uzavřena na tuto dobu.

(2) Ustanovení odstavce 1 se nepoužije přesto, že nájemce věc dál užívá, dala-li strana v přiměřené době předem najevo, že nájem skončí nebo již dříve nájem vypověděla.

§ 2231

(1) Nájem ujednaný na dobu neurčitou skončí výpovědí jednou ze stran. Jedná-li se o věc movitou, je výpovědní doba jednoměsíční, jedná-li se o věc nemovitou, je tříměsíční.

(2) Výpověď nemusí být odůvodněna; to neplatí, má-li strana právo vypovědět nájem bez výpovědní doby.

§ 2232

Porušuje-li strana zvlášť závažným způsobem své povinnosti, a tím působí značnou újmu druhé straně, má dotčená strana právo vypovědět nájem bez výpovědní doby.

§ 2233

(1) V době tří měsíců před skončením nájmu, je-li stranám den skončení nájmu znám, umožní nájemce věci, která má být znovu pronajata, zájemci o nájem přístup k věci v nezbytném rozsahu za účelem prohlídky v přítomnosti nájemce a pronajímatele; pronajímatel oznámí nájemci návštěvu v přiměřené době předem.

(2) Ustanovení § 2219 odst. 2 platí i zde.

§ 2234

Pronajímatel má právo na úhradu pohledávky vůči nájemci zadržet movité věci, které má nájemce na věci nebo v ní.

Pododdíl 2

Zvláštní ustanovení o nájmu bytu a nájmu domu
Základní ustanovení

§ 2235

(1) Zavazuje-li nájemní smlouva pronajímatele přenechat nájemci k zajištění bytových potřeb nájemce a popřípadě i členů jeho domácnosti byt nebo dům, který je předmětem nájmu, nepřihlíží se k ujednáním zkracujícím nájemcova práva podle ustanovení tohoto pododdílu.

(2) Ustanovení tohoto pododdílu se nepoužijí, přenechává-li pronajímatel nájemci byt nebo dům k rekreaci nebo jinému zjevně krátkodobému účelu.

§ 2236

(1) Bytem se rozumí místnost nebo soubor místností, které jsou částí domu, tvoří obytný prostor a jsou určeny a užívány k účelu bydlení. Ujednají-li si pronajímatel s nájemcem, že k obývání bude pronajat jiný než obytný prostor, jsou strany zavázány stejně, jako by byl pronajat obytný prostor.

(2) Skutečnost, že pronajatý prostor není určen k bydlení, nemůže být na újmu nájemci.

(3) Je-li k zajištění bytových potřeb nájemce pronajat dům, použijí se ustanovení o nájmu bytu přiměřeně.

§ 2237

Smlouva vyžaduje písemnou formu; pronajímatel však nemá právo namítnout vůči nájemci neplatnost smlouvy pro nedostatek formy.

§ 2238

Užívá-li nájemce byt po dobu tří let v dobré víře, že nájem je po právu, považuje se nájemní smlouva za řádně uzavřenou.

§ 2239

Zakázaná ujednání
Nepřihlíží se k ujednání ukládajícímu nájemci povinnost, která je vzhledem k okolnostem zjevně nepřiměřená.

Odevzdání bytu

§ 2242

(1) Není-li ujednána doba, kdy pronajímatel zpřístupní nájemci byt způsobilý k nastěhování a obývání, zpřístupní pronajímatel nájemci byt prvního dne měsíce následujícího po dni, kdy smlouva nabyla účinnosti. Byt je zpřístupněn, obdržel-li nájemce klíče a nebrání-li mu nic v přístupu do bytu.

(2) Pronajímatel se může s nájemcem dohodnout, že k obývání bude předán byt, který není způsobilý k obývání. Takové ujednání je platné, jen jsou-li zároveň ujednána zvláštní práva a povinnosti plynoucí ze zvláštní povahy bytu, včetně výše a způsobu úhrady nákladů na provedení nutných úprav.

§ 2243

Byt je způsobilý k nastěhování a obývání, odpovídá-li ujednáním ve smlouvě, a není-li nic ujednáno, je byt způsobilý k nastěhování a obývání, pokud je čistý a ve stavu, který se obvykle považuje za dobrý, a pokud je zajištěno poskytování nezbytných plnění spojených s užíváním bytu nebo s ním souvisících.

§ 2244

(1) Není-li v ujednanou dobu byt způsobilý k nastěhování a obývání nebo je-li byt ve stavu, který neodpovídá sdělení pronajímatele, má nájemce právo odmítnout se nastěhovat. Nastěhuje-li se, má právo požadovat na pronajímateli splnění smlouvy; neučiní-li tak bez zbytečného odkladu, jeho právo zaniká.

(2) Znal-li nájemce stav bytu již při uzavření smlouvy, ustanovení odstavce 1 se nepoužije. To platí i v případě, že nájemce stav bytu při uzavření smlouvy neznal, protože si jej neprohlédl, ačkoli pronajímatel včas a řádně vyzval nájemce k prohlídce.

§ 2245

Využije-li nájemce právo nenastěhovat se do bytu, není povinen platit nájemné po dobu, co vada trvá. Nastěhuje-li se, má právo na přiměřenou slevu z nájemného, dokud pronajímatel vadu neodstraní; to platí i v případě podstatné vady v poskytování plnění spojeného nebo souvisícího s užíváním bytu.

Nájemné a jiné platby

§ 2246

(1) Strany ujednají nájemné pevnou částkou. Má se za to, že se nájemné sjednává za jeden měsíc.

(2) Neujednají-li strany výši nájemného, vznikne pronajímateli právo na nájemné v takové výši, jaká je v den uzavření smlouvy v místě obvyklá pro nový nájem obdobného bytu za obdobných smluvních podmínek.

§ 2247

(1) Strany si ujednají, která plnění spojená s užíváním bytu nebo s ním související služby zajistí pronajímatel; schází-li takové ujednání, použije se ustanovení odstavce 2.

(2) Pronajímatel zajistí po dobu nájmu nezbytné služby. Má se za to, že nezbytnými službami jsou dodávky vody, odvoz a odvádění odpadních vod včetně čištění jímek, dodávky tepla, odvoz komunálního odpadu, osvětlení a úklid společných částí domu, zajištění příjmu rozhlasového a televizního vysílání, provoz a čištění komínů, případně provoz výtahu.

(3) Způsob rozúčtování cen a úhrady služeb stanoví jiný právní předpis.

(4) Strany si ujednají způsob rozúčtování cen a úhrady případných dalších služeb, není-li stanoven jiným právním předpisem nebo rozhodnutím cenového orgánu. Způsob rozúčtování musí být určen před poskytováním služby.

"""

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Shrň mi v několika větách přiložená pravidla" + law, # instruction
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True)
summary = tokenizer.batch_decode(outputs)

In [ ]:
print(summary[0].split("### Response:")[-1])


Shrnutí:
Tento dokument je součástí právního předpisu, který se zabývá nájemními smlouvy. Obsahuje několik ustanovení týkajících se nájemních smluv, včetně obecných ustanovení, ustanovení týkajících se nájemného, ustanovení týkajících se skončení nájmu a ustanovení týkajících se nájemních smluv týkajících se nájemu bytu a domu.<eos>


 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Continue the fibonnaci sequence.", # instruction
        "1, 1, 2, 3, 5, 8", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<bos>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Continue the fibonnaci sequence.

### Input:
1, 1, 2, 3, 5, 8

### Response:
13, 21, 34, 55, 89, 144<eos>


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("lora_model") # Local saving
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
[
    alpaca_prompt.format(
        "What is a famous tall tower in Paris?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<bos>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nWhat is a famous tall tower in Paris?\n\n### Input:\n\n\n### Response:\nOne of the most famous tall towers in Paris is the Eiffel Tower. It is a wrought-iron lattice tower on the Champ de Mars in Paris, France. It is named after the engineer Gustave Eiffel, whose company designed and built the tower. The tower is 324 meters (1,063 feet']

You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if True: model.push_to_hub_merged("simecek/law_summarizer", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

Unsloth: You are pushing to hub, but you passed your HF username = simecek.
We shall truncate simecek/law_summarizer to law_summarizer
Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which will take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 5.6G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 6.45 out of 12.67 RAM for saving.


100%|██████████| 28/28 [01:02<00:00,  2.23s/it]


Unsloth: Saving tokenizer...

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

...

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

pytorch_model-00004-of-00004.bin:   0%|          | 0.00/2.11G [00:00<?, ?B/s]

pytorch_model-00003-of-00004.bin:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00004.bin:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Done.
Saved merged model to https://huggingface.co/simecek/law_summarizer


### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in `llama.cpp` or a UI based system like `GPT4All`. You can install GPT4All by going [here](https://gpt4all.io/index.html).

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/u54VK8m8tk) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Zephyr DPO 2x faster [free Colab](https://colab.research.google.com/drive/15vttTpzzVXv_tJwEk-hIcQ0S9FcEWvwP?usp=sharing)
2. Llama 7b 2x faster [free Colab](https://colab.research.google.com/drive/1lBzz5KeZJKXjvivbYvmGarix9Ao6Wxe5?usp=sharing)
3. TinyLlama 4x faster full Alpaca 52K in 1 hour [free Colab](https://colab.research.google.com/drive/1AZghoNBQaMDgWJpi4RbffGM1h6raLUj9?usp=sharing)
4. CodeLlama 34b 2x faster [A100 on Colab](https://colab.research.google.com/drive/1y7A0AxE3y8gdj4AVkl2aZX47Xu3P1wJT?usp=sharing)
5. Mistral 7b [free Kaggle version](https://www.kaggle.com/code/danielhanchen/kaggle-mistral-7b-unsloth-notebook)
6. We also did a [blog](https://huggingface.co/blog/unsloth-trl) with 🤗 HuggingFace, and we're in the TRL [docs](https://huggingface.co/docs/trl/main/en/sft_trainer#accelerate-fine-tuning-2x-using-unsloth)!
7. `ChatML` for ShareGPT datasets, [conversational notebook](https://colab.research.google.com/drive/1Aau3lgPzeZKQ-98h69CCu1UJcvIBLmy2?usp=sharing)
8. Text completions like novel writing [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing)

<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a> Support our work if you can! Thanks!
</div>